# Teams

在 AutoGen 中，团队（Team）是由多个 Agent 协作完成任务的一种结构，能处理更复杂、跨领域的场景。当单一 Agent 无法应对复杂任务时，可以转而使用团队模式。也就是多智能体协作。在本节中将会说明关于 Teams 以下内容：

- 关于 Teams 的相关操作（包括创建，运行等）
- Autogen 中预设的各种 Teams
- 将对比探讨 Teams 与 Swarm 

> 0.6.0 之前的 Autogen 中的 `GroupChat` ，在最新版 0.9.0 中更改为 Team。


参考：https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/tutorial/teams.html

## 配置 Client

参考: `0-Client.ipynb`

In [1]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient

load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY") # 读取你的 OPENAI API key

# 初始化 OpenAIChatCompletionClient 客户端，连接到硅基流动平台的 Qwen3-8B 模型
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-8B",                # 指定要调用的模型名称，硅基流动平台上 Qwen 3-8B 模型
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True           
    },
)

## 预设的 Teams

Autogen 中有一些预设的 Team 常见的团队预设（preset）包括下面四种：

- **RoundRobinGroupChat**  
  代理（Agent）按照顺序依次发言，每次发言后，将消息广播给所有成员，保持共享对话上下文一致。适合需要所有成员都能看到全部对话内容的场景。

- **SelectorGroupChat**  
  每次发言后由模型评估选择下一位发言者，动态决定交流顺序。适合需要智能决策对话流程、动态分配任务的场景。

- **Swarm**  
  基于 `HandoffMessage` 转发机制，每个 Agent 可以主动将任务转交给其他 Agent，实现类似分工协作的效果。适合任务链式流转、复杂协作场景。

- **MagenticOneGroupChat**  
  通用型多 Agent 系统，适用于网页或文件驱动任务，支持跨领域、多类型任务的解决方案。适合需要多智能体协同处理多源信息的复杂

本节的操作示例将都使用 `RoundRobinGroupChat` 进行演示。

### RoundRobinGroupChat

`RoundRobinGroupChat` 是 AutoGen 中用于多智能体团队协作的核心类，实现了“轮流发言”的群聊机制。每个参与者（Agent）依次发言，消息会广播给所有成员，适合需要共享上下文的多智能体协作场景。

适用于适合需要多智能体轮流协作、共享上下文的场景。通过灵活配置终止条件、轮数和消息类型，可以满足绝大多数多智能体团队协作需求。

该类的主要特性为：

- **轮流发言**：每个 Agent 按顺序依次发言，消息广播给所有成员。
- **终止条件**：可通过关键词、最大轮数等灵活控制对话何时结束。
- **自定义消息类型**：支持扩展，适合高级多模态或自定义事件场景。
- **团队事件流**：可选，便于实时监控团队状态。


| 参数名                  | 类型                                                         | 说明                                                                                      |
|-------------------------|--------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| participants            | List[BaseChatAgent]                                          | 群聊中的参与者（Agent）列表。必须提供，且每个 Agent 名称唯一。                             |
| termination_condition   | TerminationCondition, optional                               | 群聊终止条件（如关键词、轮数等）。不设置则无限循环。                                      |
| max_turns               | int, optional                                                | 最大轮数，超过后自动停止。默认无限制。                                                    |
| runtime                 | AgentRuntime, optional                                       | 运行时配置，通常无需手动设置。                                                            |
| custom_message_types    | List[type[BaseAgentEvent \| BaseChatMessage]], optional      | 自定义消息类型列表。如果 Agent 会产生自定义消息类型，需在此声明。                         |
| emit_team_events        | bool, optional                                               | 是否通过 `run_stream()` 输出团队事件。默认 False。                                        |

**注意事项**

- `participants` 必须为 BaseChatAgent 的实例列表，且名称唯一，否则会抛出 ValueError。
- 如果团队只有一个成员，则只有该成员发言。
- 若使用自定义消息类型，需确保类型继承自 BaseAgentEvent 或 BaseChatMessage，并在 `custom_message_types` 中声明。

参考：https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat

### SelectorGroupChat (Todo)

参考：https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/selector-group-chat.html

### Swarm (Todo)

参考：https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/swarm.html

### MagenticOneGroupChat (Todo)


参考：https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/magentic-one.html

## 关于 Teams 的基础操作

下面将使用 `RoundRobinGroupChat` 讲解与演示关于 Teams 结构的相关基础操作，包含：

- 创建一个 Team (Create a Team)
- 运行一个 Team (Running a Team)
- 观察一个 Team (Observing a Team)
- 重置一个 Team (Resetting a Team)
- 停止一个 Team (Stopping a Team)
- 重启一个 Team (Resuming a Team)
- 丢弃一个 Team (Aborting a Team)
- (Single-Agent Team)

### Create a Team

我们将首先创建一个包含 两个 AssistantAgent 的 Team，并设置一个 `TextMentionTermination` 条件，当代理响应中检测到特定词语时，团队对话将自动停止。这个双代理团队实现了“反思模式”（reflection pattern），这是一种多智能体设计模式，其中一个“评论者代理”（critic agent）负责评估主代理的响应。

> 反思模式 参考: https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/design-patterns/reflection.html

通过下面的例子可以总结得出 创建一个 Team 的流程为：

- 创建好需要用到的 Agent
- 选择合适的 Team 类型
- 将 Agent 编入 Team
- 设定停止条件

In [3]:
import asyncio

# 导入 AutoGen 中的核心组件
from autogen_agentchat.agents import AssistantAgent  # 引入智能体类（对话代理）
from autogen_agentchat.base import TaskResult       # 引入任务结果类
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination  # 引入终止条件类
from autogen_agentchat.teams import RoundRobinGroupChat  # 引入轮询式群聊团队类
from autogen_agentchat.ui import Console            # 控制台界面（可选，用于调试和输出）
from autogen_core import CancellationToken          # 引入取消令牌，用于中断任务执行
from autogen_ext.models.openai import OpenAIChatCompletionClient  # OpenAI 接口模型客户端


# 创建主模型客户端（应当在实际运行前配置好 model_client）
model_client = model_client

# 创建主助手智能体（primary agent）
primary_agent = AssistantAgent(
    "primary",  
    model_client=model_client,  # 使用的语言模型客户端
    system_message="You are a helpful AI assistant.",  # 系统提示词：用于设定角色和行为
)

# 创建评论者智能体（critic agent）
critic_agent = AssistantAgent(
    "critic",  
    model_client=model_client,  # 使用同样的模型客户端
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
    # 设定为反馈审查角色，一旦认为问题已被解决就回复 "APPROVE"
)

# 定义一个终止条件：当 critic 回复中提到 “APPROVE” 时，任务终止
text_termination = TextMentionTermination("APPROVE")

# 使用 RoundRobinGroupChat 创建一个轮流发言的团队
# 团队成员包括 primary_agent 和 critic_agent，依据设定的终止条件停止循环
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],  # 成员列表
    termination_condition=text_termination  # 对话终止条件
)

### Running a Team

使用 `.run` 来运行 Teams

该团队会持续运行各个代理，直到满足终止条件。
在这个示例中，团队按照轮询顺序依次运行代理，直到在某个代理的响应中检测到关键词 “APPROVE”，从而触发终止条件。
当团队停止运行时，它会返回一个 TaskResult 对象，其中包含该团队内所有代理生成的消息记录。

In [4]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Where is the Beijing?")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Where is the Beijing?', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, the user is asking "Where is the Beijing?" I need to figure out what they\'re really looking for. First, I should confirm if they\'re asking for the location of Beijing, China, or if there\'s another context. Since "Beijing" is a proper noun and a major city, it\'s likely they want to know its geographical location.\n\nBeijing is the capital of the People\'s Republic of China. It\'s located in the northern part of the country. To give a complete answer, I should mention its coordinates, nearby cities, and maybe some key landmarks. Coordinates are approximately 39.9042° N, 116.4074° E. Nearby cities include Tianjin, which is to the southeast, and cities like Shijiazhuang and Baoding to the south. Key landmarks would be the Forbidden City, Tiananmen Square, and the Great Wall of China nea

### Observing a Team

Observing a Team 指的是在 Team 运行的过程中，实时监听和获取各个智能体发出的消息流，以实现任务观察、可视化、控制等目的。这就类似于“监听一个会议的全过程，并随时看到谁在发言、说了什么、任务进行到哪里了”。

Observe 的应用场景：

1. **调试与开发：** 你需要查看 agent 间的交互细节，确认任务是否如预期推进、终止条件是否触发。
2. **实时反馈：** 构建 UI 可视化界面，如 ChatConsole、Web UI，实时输出 agent 的回复。
3. **中途干预/控制流程：** 你可能想在某些轮次插入额外逻辑，或根据发言内容动态修改任务。


我们通过调用团队的 `run_stream()` 方法，在团队运行过程中实时获取消息流。该方法会返回一个生成器（generator），在代理生成消息时会依次产出这些消息，最后一个产出的结果是 `TaskResult` 对象，包含团队所有生成的消息记录。

`run_stream()` 方法的输入参数表格：


| 参数名                          | 类型                                          | 是否可选 | 说明                                                             |
| ---------------------------- | ------------------------------------------- | ---- | -------------------------------------------------------------- |
| `task`                       | `str` 或 `BaseChatMessage`                   | 否    | 初始任务输入，可以是字符串或消息对象，作为对话的起点。                                    |
| `observer`                   | `Callable[[BaseChatMessage], Any]` 或 `None` | 是    | 可选的观察者函数，每轮消息生成后会被调用一次。用于日志、分析或外部接口触发。                         |
| `max_loops`                  | `int` 或 `None`                              | 是    | 设置团队最多循环轮数。超过后即使未满足终止条件也会强制停止。常用于测试或防止死循环。                     |
| `max_consecutive_auto_reply` | `int` 或 `None`                              | 是    | 限制某个 agent 连续自动回复的最大次数，用于防止某个 agent 无限自言自语（仅当只有一个 agent 时有意义）。 |
| `initial_messages`           | `List[BaseChatMessage]` 或 `None`            | 是    | 用于设置任务开始前的“上下文历史消息”，相当于提供一个已有的对话背景。                            |
| `sender`                     | `str` 或 `None`                              | 是    | 指定任务开始时的首个发言者，默认为团队列表中的第一个 agent。                              |
| `start_message`              | `BaseChatMessage` 或 `None`                  | 是    | 明确指定第一条发言的消息，常用于精细控制起始内容（高级用法）。                                |


`TaskResult` 字段表格（对话终止后返回的最终结构）

| 字段名             | 类型                         | 说明                                  |
| --------------- | -------------------------- | ----------------------------------- |
| `messages`      | `List[BaseChatMessage]`    | 本次任务的完整消息历史列表，按时间顺序排列。              |
| `final_message` | `BaseChatMessage` 或 `None` | 最后一个 agent 发出的消息（终止前一轮）。            |
| `stop_reason`   | `str`                      | 任务终止的原因（如关键词命中、最大轮数、外部中断等）。         |
| `metrics`       | `dict`                     | 可选。包含运行时间、token 使用、轮数等评估指标（依赖具体实现）。 |


`BaseChatMessage` 字段表格（团队中每一轮对话的基本单位）

| 字段名          | 类型         | 说明                                            |
| ------------ | ---------- | --------------------------------------------- |
| `sender`     | `str`      | 消息发送者的名称（对应某个 agent 的 `name`）。                |
| `content`    | `str`      | 消息的正文内容。通常是 agent 生成的自然语言文本。                  |
| `created_at` | `datetime` | 消息生成的时间戳（可用于排序或记录）。                           |
| `metadata`   | `dict`     | 附加的元数据，可能包含调用细节、日志、评估标签等。具体内容取决于 agent 和模型配置。 |



In [6]:
# 当在脚本中运行时，建议将主逻辑写在 async main 函数中，并用 asyncio.run(main()) 启动。
# 下面演示如何重置团队并实时观察团队运行过程。

await team.reset()  # 重置团队状态，为新任务做准备（清空历史消息等）

# 通过 run_stream() 方法启动团队任务，并实时获取每条消息
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        # 如果是最终的 TaskResult，说明团队任务已结束
        print("Stop Reason:", message.stop_reason)  # 输出终止原因
    else:
        # 其他为团队成员（Agent）生成的中间消息
        print(message)  # 实时输出每条消息内容

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=None metadata={} content='\nOkay, the user wants a short poem about the fall season. Let me start by brainstorming some key elements of fall. There\'s the changing leaves, cooler weather, harvest time, maybe pumpkins and apples. Also, the feeling of transition, like the end of summer and the approach of winter.\n\nI should think about the structure. A short poem could be four stanzas with four lines each, maybe a rhyme scheme. Let me go with AABB or ABAB for simplicity. Let\'s see... First stanza could introduce the visual aspects: leaves changing color, maybe the wind. \n\nSecond stanza might focus on the harvest, like farmers, fruits, and the earth. Third stanza could include elements like the sun\'s angle, shorter days, and maybe animals preparing for winter. The last stanza should wrap up with the overall feeling of fall, perhaps a sense o

正如上面的示例所展示的，你可以通过检查 `stop_reason` 属性来判断团队任务为何终止。
> `stop_reason` 是你评估任务状态、控制流程、调试错误、打标签的关键字段，每次任务结束都应该检查它。

### Resetting a Team

Resetting a Team 就是重置团队状态，使其像“刚初始化时”一样干净，适合开始一个全新的任务。

你可以通过调用 `reset()` 方法来重置团队，此方法会清除整个团队的状态，包括每个智能体的内部状态（如历史消息）。它会调用每个 agent 的 on_reset() 方法，确保 agent 自己也能清空缓存、状态变量等。所以，所以调用是异步的，必须用 `await`。

| 什么时候该用？             | 原因                                    |
| ------------------- | ------------------------------------- |
| ❌ 如果不 reset，而任务又不相关 | agent 会“记住”前一次任务的上下文，导致行为错乱或话题跳脱      |
| ✅ 如果你要换一个主题         | 比如从“写诗”变成“写代码”，就应该 reset              |
| ✅ 如果你在做批量任务         | 每次循环处理新输入之前，都应调用 `await team.reset()` |


In [5]:
await team.reset()  # 重置团队状态，使其恢复到初始状态，清空历史消息，方便开始下一个全新任务

### Stopping a Team

如 创建Team  ，在 AutoGen 中，团队任务默认会根据内部条件终止，比如：

- 指定关键词被提及 → `TextMentionTermination("APPROVE")`
- 超过最大轮数 → `max_turns`
- 只有一个 agent → 自动终止

但是有时候你希望 从外部手动控制对话终止 —— 比如：

> 当你按下按钮、收到某个信号、满足某个外部系统状态时，让对话优雅地停止。

此时我们就需要使用 `ExternalTermination`。它是一个外部控制的终止条件，允许你在运行过程中从外部发起停止信号。默认状态是“未终止”，调用 `.set()`方法后表示“请求终止”。




| 特性            | 自动终止（如关键词） | `ExternalTermination`（外部控制） |
| ------------- | ---------- | --------------------------- |
| 是否依赖 agent 内容 | 是（依赖对话文本）  | 否（手动调用）                     |
| 是否立即终止        | 是          | 否，需要当前 agent 说完话后           |
| 是否适合异步控制流程    | ❌ 不灵活      | ✅ 可动态触发、取消、远程控制             |


In [7]:
# 创建一个带有外部终止条件的新团队。
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # 使用按位或运算符组合多个终止条件。
)

# 以后台任务方式运行团队。
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# 等待一段时间。
await asyncio.sleep(0.1)

# 停止团队（触发外部终止条件）。
external_termination.set()

# 等待团队任务结束。
await run

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- ThoughtEvent (primary) ----------

Okay, the user wants another short poem about the fall season. Let me think... They already asked for one called "Autumn's Whisper" and got a response. Now they're asking again, maybe they want a different angle or style.

Hmm, maybe they're looking for something more vivid or with different imagery. Let me consider the elements of fall: leaves changing color, cooler weather, harvest time, maybe Halloween, the approach of winter. Should I focus on nature or include human activities?

In the previous poem, there were elements like leaves, harvest moon, farmers, geese, cinnamon, apple cider. Maybe this time I can add more sensory details or personify the season more. Let me try to use different metaphors. 

What about the feeling of transition? Fall as a bridge between summer and winter. Maybe mention the colors, the crisp air, the sense of closure. Or perhaps 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\nOkay, the user wants another short poem about the fall season. Let me think... They already asked for one called "Autumn\'s Whisper" and got a response. Now they\'re asking again, maybe they want a different angle or style.\n\nHmm, maybe they\'re looking for something more vivid or with different imagery. Let me consider the elements of fall: leaves changing color, cooler weather, harvest time, maybe Halloween, the approach of winter. Should I focus on nature or include human activities?\n\nIn the previous poem, there were elements like leaves, harvest moon, farmers, geese, cinnamon, apple cider. Maybe this time I can add more sensory details or personify the season more. Let me try to use different metaphors. \n\nWhat about the feeling of transition? Fall as a 

当然还有其他的中止条件

除了 TextMessageTermination，还有很多终止条件可选：

- `MaxTurnsTermination`：最大轮数后终止；

- `ExternalTermination`：外部主动触发终止；

- `TextMentionTermination`：当消息中出现某关键词时终止；

- 组合多种条件（用逻辑运算符 |、& 组合）等。

会在后面的章节做详细的说明


### Resuming a Team

1. Team 是有记忆的（stateful）
  - 所有的历史对话（`BaseChatMessage`）和轮次状态都会保留；
  - 只有不调用 `reset()`，就可以继续互动
2. 可以继续上一次中断的对话
  - 直接再次调用 `run()` 或 `run_stream()`；
  - 如果你不传入新的 `task`，团队会自动认为你要“接着聊”。
3. `RoundRobinGroupChat` 会记住轮到谁发言
  - 比如第一次轮到了 `primary`，第二次就会从 `critic` 开始；
  - 这样确保对话是“顺畅连贯”的，避免重复。

继续完成上一个 Part 未完成的任务。

In [8]:
await Console(team.run_stream())  # 恢复team，继续上一次未完成的任务（从中断处继续运行）

---------- ThoughtEvent (critic) ----------

Okay, let's see. The user wants a short poem about the fall season again. They previously submitted one called "Autumn's Whisper" and received feedback. Now they've given a new version titled "Whispers of Autumn" and I need to evaluate it.

First, I'll check the imagery and flow. The new poem uses phrases like "Leaves blush in hues of flame," which is vivid. The rhyme scheme is still AABB, which is consistent. The lines about pumpkins and stars are nice, adding a touch of storytelling.

Next, looking at word choice. The original feedback mentioned some abstract phrases. In the second version, "the earth exhales in amber" is a good metaphor. However, "caskets of pumpkins" might be a bit too metaphorical. Maybe that's okay, but I should consider if it's clear enough. Also, "scripting tales of the night" is personifying the stars, which is a nice touch, but perhaps could be more concrete.

Sensory balance: The poem includes visual elements (blu

TaskResult(messages=[ThoughtEvent(source='critic', models_usage=None, metadata={}, content='\nOkay, let\'s see. The user wants a short poem about the fall season again. They previously submitted one called "Autumn\'s Whisper" and received feedback. Now they\'ve given a new version titled "Whispers of Autumn" and I need to evaluate it.\n\nFirst, I\'ll check the imagery and flow. The new poem uses phrases like "Leaves blush in hues of flame," which is vivid. The rhyme scheme is still AABB, which is consistent. The lines about pumpkins and stars are nice, adding a touch of storytelling.\n\nNext, looking at word choice. The original feedback mentioned some abstract phrases. In the second version, "the earth exhales in amber" is a good metaphor. However, "caskets of pumpkins" might be a bit too metaphorical. Maybe that\'s okay, but I should consider if it\'s clear enough. Also, "scripting tales of the night" is personifying the stars, which is a nice touch, but perhaps could be more concret

In [8]:
await Console(team.run_stream(task="将这首诗用中文唐诗风格写一遍。"))

---------- TextMessage (user) ----------
将这首诗用中文唐诗风格写一遍。
---------- ThoughtEvent (primary) ----------

好的，用户让我把之前写的关于秋天的英文诗转换成中文唐诗风格。首先，我需要理解唐诗的特点。唐诗通常讲究格律，对仗工整，押韵，意境深远，语言凝练。所以，我得先分析原诗的结构和主题，再转换成符合唐诗形式的中文。

原诗分为四段，每段四行，押韵可能不太严格，但整体有节奏感。主题集中在秋天的变迁、自然景色、收获季节以及时间的流逝。我需要确保这些元素在唐诗中都能体现出来。

接下来，考虑如何将每一句翻译或意译成唐诗的句子。例如，第一句“Whispers of change on the wind take flight”可以转化为“风送秋声悄”，既有风的意象，又表达了季节变化的感觉。第二句“Leaves dance in hues of fire and gold”需要描绘落叶的颜色，可能用“叶落如红金”来简洁表达。

第三句提到“The earth prepares for slumber”，这里要传达大地进入休眠状态，可能用“大地入眠期”或者更诗意的“大地敛华光”。第四句“As twilight spills its amber light”可以翻译为“暮色洒金光”，保持画面感。

第二段中的“Crisp apples hang in the orchard's embrace”需要描绘苹果的香气和丰收，或许用“霜果垂枝上”来表现。而“Pumpkins grin in the fields with grace”可能转化为“南瓜笑满田”，用拟人手法增加生动性。

第三段的“Harvest time hums in the fields of wheat”可以处理为“麦浪涌丰收”，既表现了收获的喜悦又押韵。“The chill of dawn lingers in the air”则可能需要更简洁的表达，比如“晨寒浸晓风”。

最后一段“Autumn, a painter, strokes the world with care”可以转化为“秋为丹青手”，用比喻传达秋天的创造力。“Crimson, amber, and embered light”可能需要用更符合中文意境的

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='将这首诗用中文唐诗风格写一遍。', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content="\n好的，用户让我把之前写的关于秋天的英文诗转换成中文唐诗风格。首先，我需要理解唐诗的特点。唐诗通常讲究格律，对仗工整，押韵，意境深远，语言凝练。所以，我得先分析原诗的结构和主题，再转换成符合唐诗形式的中文。\n\n原诗分为四段，每段四行，押韵可能不太严格，但整体有节奏感。主题集中在秋天的变迁、自然景色、收获季节以及时间的流逝。我需要确保这些元素在唐诗中都能体现出来。\n\n接下来，考虑如何将每一句翻译或意译成唐诗的句子。例如，第一句“Whispers of change on the wind take flight”可以转化为“风送秋声悄”，既有风的意象，又表达了季节变化的感觉。第二句“Leaves dance in hues of fire and gold”需要描绘落叶的颜色，可能用“叶落如红金”来简洁表达。\n\n第三句提到“The earth prepares for slumber”，这里要传达大地进入休眠状态，可能用“大地入眠期”或者更诗意的“大地敛华光”。第四句“As twilight spills its amber light”可以翻译为“暮色洒金光”，保持画面感。\n\n第二段中的“Crisp apples hang in the orchard's embrace”需要描绘苹果的香气和丰收，或许用“霜果垂枝上”来表现。而“Pumpkins grin in the fields with grace”可能转化为“南瓜笑满田”，用拟人手法增加生动性。\n\n第三段的“Harvest time hums in the fields of wheat”可以处理为“麦浪涌丰收”，既表现了收获的喜悦又押韵。“The chill of dawn lingers in the air”则可能需要更简洁的表达，比如“晨寒浸晓风”。\n\n最后一段“Autumn, a

下面演示 team 是有状态的。可以通过 `run()`或`run_stream()` 进行追问。

In [11]:
await team.reset()

# 第一轮任务
await Console(team.run_stream(task="解释人工智能"))

---------- TextMessage (user) ----------
解释人工智能
---------- ThoughtEvent (primary) ----------

嗯，用户让我解释人工智能，我需要从基础开始。首先，应该明确人工智能的定义，它不是魔法，而是科学技术的产物。用户可能对AI有一定的了解，但需要更系统的解释。他们可能想知道AI是什么，主要有哪些类型，以及应用领域。

接下来，要区分人工智能和机器学习。用户可能容易混淆这两个概念，所以需要清楚说明机器学习是AI的一个子领域，主要依赖数据训练模型。然后是深度学习，作为机器学习的分支，特别擅长处理图像和语音数据。

还需要提到AI的不同应用，比如语音识别、自动驾驶、医疗诊断等等。这些例子能让用户更直观地理解AI的实际用途。另外，可能涉及AI的伦理和挑战，比如隐私、偏见、就业影响等，这些是用户可能关心但不太了解的方面。

要注意不要使用太多专业术语，保持解释简单易懂。可能需要用比喻，比如把AI比作人脑的延伸，帮助用户更好理解。还要考虑用户可能的深层需求，比如他们是否想了解AI对个人生活的影响，或者如何开始学习AI。

最后，检查是否有遗漏的重要点，比如AI的发展历史，或者当前的热门技术如自然语言处理、计算机视觉。确保覆盖全面，同时保持回答的流畅和自然。

---------- TextMessage (primary) ----------


人工智能（Artificial Intelligence，简称AI）是计算机科学的一个分支，旨在开发能够模拟人类智能行为的系统或机器。其核心目标是让机器具备感知、学习、推理、决策、创造等能力，从而在各种任务中表现出类似人类的智慧。

### 核心概念
1. **模拟人类智能**  
   AI通过算法和数据处理，模仿人类的感知、思考和行动能力。例如，语音识别系统可以“听懂”人类语言，图像识别系统可以“看懂”照片内容。

2. **技术基础**  
   AI依赖于 **机器学习**（从数据中学习规律）和 **深度学习**（使用多层神经网络模拟人脑处理信息的方式）。这些技术使计算机能够通过大量数据自主优化性能，而无需明确编程。

3. **广泛的应用领域**  
   - **语音与语言处理**：如智能客服（Siri、Alexa）、翻译工具（Google

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='解释人工智能', type='TextMessage'), ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\n嗯，用户让我解释人工智能，我需要从基础开始。首先，应该明确人工智能的定义，它不是魔法，而是科学技术的产物。用户可能对AI有一定的了解，但需要更系统的解释。他们可能想知道AI是什么，主要有哪些类型，以及应用领域。\n\n接下来，要区分人工智能和机器学习。用户可能容易混淆这两个概念，所以需要清楚说明机器学习是AI的一个子领域，主要依赖数据训练模型。然后是深度学习，作为机器学习的分支，特别擅长处理图像和语音数据。\n\n还需要提到AI的不同应用，比如语音识别、自动驾驶、医疗诊断等等。这些例子能让用户更直观地理解AI的实际用途。另外，可能涉及AI的伦理和挑战，比如隐私、偏见、就业影响等，这些是用户可能关心但不太了解的方面。\n\n要注意不要使用太多专业术语，保持解释简单易懂。可能需要用比喻，比如把AI比作人脑的延伸，帮助用户更好理解。还要考虑用户可能的深层需求，比如他们是否想了解AI对个人生活的影响，或者如何开始学习AI。\n\n最后，检查是否有遗漏的重要点，比如AI的发展历史，或者当前的热门技术如自然语言处理、计算机视觉。确保覆盖全面，同时保持回答的流畅和自然。\n', type='ThoughtEvent'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=22, completion_tokens=896), metadata={}, content='\n\n人工智能（Artificial Intelligence，简称AI）是计算机科学的一个分支，旨在开发能够模拟人类智能行为的系统或机器。其核心目标是让机器具备感知、学习、推理、决策、创造等能力，从而在各种任务中表现出类似人类的智慧。\n\n### 核心概念\n1. **模拟人类智能**  \n   AI通过算法和数据处理，模仿人类的感知、思考

In [12]:
# 第二轮续问，不传新任务
await Console(team.run_stream())

---------- ThoughtEvent (primary) ----------

好的，用户发送了“APPROVE”这个词。首先，我需要确认用户想要表达的意思。可能是在之前的对话中，他们需要我批准某个内容，或者在处理某个请求时，希望得到我的认可。

回顾之前的对话，用户第一次询问了关于人工智能的解释，我给出了详细的回答，包括核心概念、类型、技术原理和伦理挑战。用户可能对我的回答满意，但需要进一步确认或者有后续问题。不过“APPROVE”通常是一个动词，可能用户是在测试我的反应，或者他们认为回答已经足够，需要我确认。

也有可能用户是在使用某种命令或快捷方式，比如在某些系统中，输入“APPROVE”可能有特定的功能。但考虑到用户之前的问题，更可能的是他们希望得到某种形式的批准或确认。我需要检查是否有任何上下文缺失，或者是否有未完成的请求需要处理。

另外，用户可能是在询问关于AI的批准流程，比如在开发AI应用时所需的法律或政策批准，但之前的问题并未涉及这一点。因此，回复应该保持友好，确认他们是否需要进一步的帮助，同时保持开放式的沟通。

还要注意用户可能是想进行某种互动，比如模拟对话中的确认步骤，或者他们误用了这个词。因此，我的回应应该礼貌，并询问是否有具体的事情需要处理，比如审批某个文档、项目，或者是否需要更详细的信息。同时，避免做出假设，确保用户明确表达需求。

---------- TextMessage (primary) ----------


您是想让我批准某个内容或进行某种操作吗？请提供更多具体信息，我会尽力协助您。 😊


TaskResult(messages=[ThoughtEvent(source='primary', models_usage=None, metadata={}, content='\n好的，用户发送了“APPROVE”这个词。首先，我需要确认用户想要表达的意思。可能是在之前的对话中，他们需要我批准某个内容，或者在处理某个请求时，希望得到我的认可。\n\n回顾之前的对话，用户第一次询问了关于人工智能的解释，我给出了详细的回答，包括核心概念、类型、技术原理和伦理挑战。用户可能对我的回答满意，但需要进一步确认或者有后续问题。不过“APPROVE”通常是一个动词，可能用户是在测试我的反应，或者他们认为回答已经足够，需要我确认。\n\n也有可能用户是在使用某种命令或快捷方式，比如在某些系统中，输入“APPROVE”可能有特定的功能。但考虑到用户之前的问题，更可能的是他们希望得到某种形式的批准或确认。我需要检查是否有任何上下文缺失，或者是否有未完成的请求需要处理。\n\n另外，用户可能是在询问关于AI的批准流程，比如在开发AI应用时所需的法律或政策批准，但之前的问题并未涉及这一点。因此，回复应该保持友好，确认他们是否需要进一步的帮助，同时保持开放式的沟通。\n\n还要注意用户可能是想进行某种互动，比如模拟对话中的确认步骤，或者他们误用了这个词。因此，我的回应应该礼貌，并询问是否有具体的事情需要处理，比如审批某个文档、项目，或者是否需要更详细的信息。同时，避免做出假设，确保用户明确表达需求。\n', type='ThoughtEvent'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=664, completion_tokens=327), metadata={}, content='\n\n您是想让我批准某个内容或进行某种操作吗？请提供更多具体信息，我会尽力协助您。 😊', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [13]:
# 第三轮追问
await Console(team.run_stream())

---------- ThoughtEvent (critic) ----------

好的，用户之前让我解释人工智能，然后我回复了APPROVE。现在用户又发来一条消息，询问是否需要他们批准某个内容或操作，并指出我可能需要更多信息。看起来用户可能是误解了我的之前的回复，或者以为需要他们进行某种操作。

首先，我需要确认用户的问题意图。用户可能是在测试我的响应，或者确实有未明确说明的需求。之前的解释已经覆盖了AI的基本概念、类型、应用、技术原理和挑战，但用户现在可能希望更深入的讨论，或者有特定的应用场景需要解决。

接下来，我要分析用户可能的深层需求。他们可能对AI的某个方面感兴趣，比如伦理问题、技术具体实现，或者实际应用案例。或者他们可能想了解如何将AI应用于自己的项目，需要进一步的指导。但用户没有具体说明，所以我需要开放性地回应，鼓励他们提出更详细的问题。

另外，用户提到“请提供更多具体信息，我会尽力协助您”，这可能意味着他们希望我调整或补充之前的回答。我的原始回答已经比较全面，但或许在某些部分不够深入，或者他们希望得到更结构化的信息，比如分点、图示或实际例子。

我需要确保回应友好且富有帮助性，避免让用户感到未被理解。同时，保持简洁，用轻松的语气邀请他们进一步提问。可能需要检查是否有遗漏的部分，或者是否需要以不同的方式呈现信息，比如使用更简明的语言或补充更多细节。

最后，用户可能希望确认他们是否可以继续深入讨论AI的某个子领域，或者是否有其他问题需要解答。因此，我的回应应促进进一步的交流，同时明确表明我可以根据他们的需求调整回答。

---------- TextMessage (critic) ----------


感谢您的澄清！ 😊  
我的原始回答是基于您最初的问题“解释人工智能”而提供的简明概述，如果您希望我进一步调整内容（例如添加更多细节、用更通俗的语言、或补充具体案例），或者有其他与AI相关的问题需要解答，请随时告诉我！  
我随时准备根据您的需求提供更精准的反馈或协助。


TaskResult(messages=[ThoughtEvent(source='critic', models_usage=None, metadata={}, content='\n好的，用户之前让我解释人工智能，然后我回复了APPROVE。现在用户又发来一条消息，询问是否需要他们批准某个内容或操作，并指出我可能需要更多信息。看起来用户可能是误解了我的之前的回复，或者以为需要他们进行某种操作。\n\n首先，我需要确认用户的问题意图。用户可能是在测试我的响应，或者确实有未明确说明的需求。之前的解释已经覆盖了AI的基本概念、类型、应用、技术原理和挑战，但用户现在可能希望更深入的讨论，或者有特定的应用场景需要解决。\n\n接下来，我要分析用户可能的深层需求。他们可能对AI的某个方面感兴趣，比如伦理问题、技术具体实现，或者实际应用案例。或者他们可能想了解如何将AI应用于自己的项目，需要进一步的指导。但用户没有具体说明，所以我需要开放性地回应，鼓励他们提出更详细的问题。\n\n另外，用户提到“请提供更多具体信息，我会尽力协助您”，这可能意味着他们希望我调整或补充之前的回答。我的原始回答已经比较全面，但或许在某些部分不够深入，或者他们希望得到更结构化的信息，比如分点、图示或实际例子。\n\n我需要确保回应友好且富有帮助性，避免让用户感到未被理解。同时，保持简洁，用轻松的语气邀请他们进一步提问。可能需要检查是否有遗漏的部分，或者是否需要以不同的方式呈现信息，比如使用更简明的语言或补充更多细节。\n\n最后，用户可能希望确认他们是否可以继续深入讨论AI的某个子领域，或者是否有其他问题需要解答。因此，我的回应应促进进一步的交流，同时明确表明我可以根据他们的需求调整回答。\n', type='ThoughtEvent'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=706, completion_tokens=426), metadata={}, content='\n\n感谢您的澄清！ 😊  \n我的原始回答是基于您最初的问题“解释人工智能”而提供的简明概述，如果您希望我进一步调整内容（例如添加更多细节、用更通俗的语言、或补充具体案例），或者有其他与AI相关的问题需要解答，请随时告诉

### Aborting a Team

Aborting 意味着“立即中断” Team 正在执行的任务。它是通过传入一个 `CancellationToken` 给 `run()` 或 `run_stream()` 方法的 `cancellation_token` 参数实现的；当你调用该 `CancellationToken` 的 `cancel()` 方法时，团队会立刻停止当前任务；



Aborting a Team 与之前提到的 Stopping a Team 不同，停止是等当前agent发言结束后优雅终止，而中止是立即抛出异常，强制停止。

> 当 team 被中止（aborted）时，调用方会收到一个 `CancelledError` 异常。


| 特性   | Stopping Team（停止）    | Aborting Team（中止）               |
| ---- | -------------------- | ------------------------------- |
| 触发方式 | 外部调用终止条件的 `set()`    | 调用 `CancellationToken.cancel()` |
| 生效时机 | 当前 agent 轮次结束后       | 立即生效，马上停止                       |
| 任务状态 | 正常结束，返回 `TaskResult` | 抛出 `CancelledError` 异常          |
| 适用场景 | 需要优雅退出，保证数据完整        | 需要强制终止，比如超时或异常处理                |


In [14]:
# 创建一个取消令牌（CancellationToken），用于后续中止团队任务
cancellation_token = CancellationToken()

# 使用 asyncio.create_task 启动团队任务，作为后台协程运行
run = asyncio.create_task(
    team.run(
        task="总结上面你回答的内容",  # 任务内容：将诗歌翻译成西班牙语
        cancellation_token=cancellation_token,  # 传入取消令牌，便于外部随时中止
    )
)。

# 主动调用 cancel() 方法，中止团队任务（会立即抛出 CancelledError）
cancellation_token.cancel()

try:
    result = await run  # 由于任务被取消，这里会抛出 CancelledError 异常
except asyncio.CancelledError:
    print("Task was cancelled.")  # 捕获异常并输出提示

Task was cancelled.


### Single-Agent Team

Single-Agent Team 是指只包含一个智能体的团队，比如只用一个 `AssistantAgent`。通过把这个智能体放入团队（比如 `RoundRobinGroupChat`）中，可以让它循环执行多步任务，直到终止条件满足。这允许智能体多轮调用工具、调用自身，形成“闭环”，而不是单步返回结果。


和直接调用智能体的区别

| 特性   | 单智能体团队运行         | 单智能体直接运行   |
| ---- | ---------------- | ---------- |
| 执行方式 | 多轮循环调用，直到满足终止条件  | 仅执行一步，返回结果 |
| 适用场景 | 需要多轮交互、工具调用、循环任务 | 简单单步推理、回答  |
| 终止控制 | 通过团队的终止条件管理      | 一步执行结束     |


使用这种模式的意义：

- 可以让单智能体以团队模式运作，利用团队管理的多轮对话和终止机制；
- 方便扩展，比如后续再加入更多智能体，实现复杂协作；
- 统一接口，便于调试和管理。

下面是一个应用示例：

**任务：** 用一个工具对数字进行累加，直到数字达到 10。

- 智能体每轮调用工具将数字加一；
- 当数字达到 10 时，智能体发送结束消息，触发终止条件；
- 团队结束运行。

In [9]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = model_client  # 假设 model_client 已在前面初始化

# 创建一个用于数字递增的工具函数
def increment_number(number: int) -> int:
    """将输入数字加1并返回"""
    return number + 1

# 创建一个工具型智能体，能够调用 increment_number 工具
looped_assistant = AssistantAgent(
    "looped_assistant",
    model_client=model_client,
    tools=[increment_number],  # 注册工具函数
    system_message="You are a helpful AI assistant, use the tool to increment the number.",  # 设定系统提示词
)

# 设定终止条件：当 looped_assistant 回复文本消息时终止任务
termination_condition = TextMessageTermination("looped_assistant")

# 创建一个只包含单智能体的团队，并设置终止条件
team = RoundRobinGroupChat(
    [looped_assistant],
    termination_condition=termination_condition,
)

# 运行团队任务：让智能体将数字5递增到10，并实时输出每条消息
async for message in team.run_stream(task="Increment the number 5 to 10."):  # type: ignore
    print(type(message).__name__, message)

# 关闭模型客户端，释放

TextMessage source='user' models_usage=None metadata={} content='Increment the number 5 to 10.' type='TextMessage'
ThoughtEvent source='looped_assistant' models_usage=None metadata={} content='\n\n' type='ThoughtEvent'
ToolCallRequestEvent source='looped_assistant' models_usage=RequestUsage(prompt_tokens=194, completion_tokens=336) metadata={} content=[FunctionCall(id='0197bef2f3ecccefe7b385a812d37cf4', arguments=' {"number": 5}', name='increment_number')] type='ToolCallRequestEvent'
ToolCallExecutionEvent source='looped_assistant' models_usage=None metadata={} content=[FunctionExecutionResult(content='6', name='increment_number', call_id='0197bef2f3ecccefe7b385a812d37cf4', is_error=False)] type='ToolCallExecutionEvent'
ToolCallSummaryMessage source='looped_assistant' models_usage=None metadata={} content='6' type='ToolCallSummaryMessage'
ThoughtEvent source='looped_assistant' models_usage=None metadata={} content='\n\n' type='ThoughtEvent'
ToolCallRequestEvent source='looped_assistant

## Reference

1. https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/tutorial/teams.html
2. https://zhuanlan.zhihu.com/p/25117231243